In [ ]:
from sympy import *
import numpy as np
M = symbols('M', real=True, positive=True)
def Continue_Function(Z_line, Max_or_Min):
    buffer = np.copy(Z_line)
    buffer = buffer[0:len(buffer) - 1] #элемент столбца B не учитывается

    if Max_or_Min == 1:   #минимум-в строке функционала нет положительных коэффициентов
        for i in buffer:
            if i > 0:
                return True

    if Max_or_Min == 2: #максимум-в строке функционала нет отрицательных коэффициентов
        for i in buffer:
            if i < 0:
                return True

    return False
def z_line_function(Z, X, B, Signs):

    Rn, Sn, MS = [], [], []

    for i in range(Signs.size):
        if Signs[i] == '=' or Signs[i] == '>=':
            Rn.append(i)
        if Signs[i] == '>=' or Signs[i] == '<=':
            Sn.append(i)
        if Signs[i] == '>=':
            MS.append(i)

    Z_line = []

    Xjisum = 0
    Bsum = 0
    for i in range(X[0].size + 1 + len(Rn) + 1):
        if i < X[0].size:
            for j in Rn: #складываются все переменные xn из строк, содержащих R
                Xjisum = Xjisum + X[j][i]
            Z_line.append(Xjisum*M - Z[i])#для 'Rn', вычисляется сумма коэффициентов в соответствующих строках матрицы X для каждой переменной. Затем эта сумма умножается на переменную M  и вычитается из соответствующего элемента вектора Z.
            Xjisum = 0
        elif i >= X[0].size and i < (X[0].size + 1):
            for j in Sn:
                if j in MS:
                    Z_line.append(-M) # для Sn соответствующем элементу вектора Z_line присваивается -M
                else:
                    Z_line.append(0) #Для остальных переменных (не входящих в 'Rn' и 'Sn') в Z_line добавляется 0.
        elif i >= (X[0].size + 1) and i < (X[0].size + 1 + len(Rn)): #зайдет сюда два раза (по примеру): R1, R2
            Z_line.append(0)
        elif i >= (X[0].size + 1 + len(Rn)):
            for j in Rn:
                Bsum = Bsum + B[j]#вычисляется сумма свободных членов B, соответствующих этим ограничениям. Затем сумма умножается на M и добавляется в соответствующий элемент Z_line
            Z_line.append(Bsum*M)

    return Z_line
def canonization(Z, X, B, Signs):

    matrix = np.copy(X)
    B_vector = np.array([np.copy(B)], dtype= float)
    Z_line = z_line_function(Z, X, B, Signs)

    Rn, Sn, MS = [], [], []
    for i in range(Signs.size):
        if Signs[i] == '=' or Signs[i] == '>=':
            Rn.append(i)
        if Signs[i] == '>=' or Signs[i] == '<=':
            Sn.append(i)
        if Signs[i] == '>=':
            MS.append(i)

    Smatrix = np.eye(matrix.shape[0]) #квадратная единичная матрица с количеством строк от matrix
    Smatrix_buff = np.eye(matrix.shape[0], 1) #буффер

    for i in Sn:
        if i in MS:
            Smatrix[i] = Smatrix[i]*(-1) #домножение на -1 всех S, имеющих в Zстроке -M,т.к М по задаче большое положительное число
    for i in range(Signs.size):
        if i in Sn:
            Smatrix_buff = np.c_[ Smatrix_buff, Smatrix[i] ] #заполнение буффера нужными строками S

    Smatrix_buff = np.delete(Smatrix_buff, [0], axis=1) #избавление от лишнего столбца буффера

    #добавление S к матрице
    matrix = np.concatenate((matrix, Smatrix_buff), axis=1)


    Rmatrix = np.eye(matrix.shape[0]) #квадратная единичная матрица с количеством строк от matrix
    Rmatrix_buff = np.eye(matrix.shape[0], 1) #буффер

    for i in range(Signs.size):
        if i in Rn:
            Rmatrix_buff = np.c_[ Rmatrix_buff, Rmatrix[i] ] #заполнение буффера нужными строками R

    Rmatrix_buff = np.delete(Rmatrix_buff, [0], axis=1) #избавление от лишнего первого столбца

    #добавление R к матрице
    matrix = np.concatenate((matrix, Rmatrix_buff), axis=1)

    # добавление столбца свободных коэффициентов справа от матрцицы
    matrix = np.concatenate((matrix,B_vector.T), axis=1)

    #присваивание индексов начального базиса Rn + (Sn в строках которых нет R)
    #в примере это номера 5,6,4: в массиве как 4, 5, 3 индексация
    basis = []
    for i in range(len(Z_line)-1 - len(Rn), len(Z_line)-1): #номера Rn в строке
            basis.append(i) #номера Rn идут после xn и Sn

    for i in range(len(X[0]), len(Z_line)-1 - len(Rn)):#номера Sn в строке, имеющие 0 в Z_line.
        if Z_line[i] == 0:
            basis.append(i)

    return  matrix, Z_line, basis
def get_str_resh(Z_resh, matrix_input):
    matrix = np.copy(matrix_input)

    B = matrix[:, matrix.shape[1] - 1] # столбец "B"

    Resh_stolb = matrix[:, Z_resh] # решающий столбец

    # если элемент в решающем столбце оказался равным 0, то чтобы на него не делить "B", пусть там будет стоять число -1.
    #Тогда при делении обязательно получится отрицательное частное и оно далее учитываться не будет.
    Resh_stolb[Resh_stolb == 0] = -1

    alpha = B / Resh_stolb # частные элементов "B" столбца и решающего-для того чтобы найти минимальный и получить решающую строку

    #все элементы из массива alpha, что <= 0, переприсваиваются в бесконечность.
    alpha = np.where(alpha > 0, alpha, np.inf)

    #поиск минимального положительного частного "B" столбца и решающего
    str_resh = alpha.argmin()

    #если минимальное частное - бесконечность, значит все alpha изначально оказались <= 0.
    if alpha[str_resh] == np.inf:
        raise Exception("Все частные элементов B столбца и решающего оказались <= 0.")
    else:
        return str_resh
def get_Z_line(matrix, Z_line_old, Z_resh, str_resh, Max_or_Min): #подсчёт Z строки

    Z_line = Z_line_old.copy()
    Z_line[Z_resh] = 0

    for i in range(len(Z_line)):
        if i != Z_resh:
            Z_line[i] = Z_line_old[i] - (matrix[str_resh][i] * Z_line_old[Z_resh]) / matrix[str_resh][Z_resh] # i элемент в новой z строке=эл-т i исхдной строки z -(эл-т матрицы в реш строке и столбце i умножаем на эл-т исходной z,которая соотвествует реш эл-ту)/реш эл-т

    if Max_or_Min == 1: #проверяем, соответствует ли новое значение функции цели по критерию максимума или минимума
        if Z_line[len(Z_line)-1] > Z_line_old[len(Z_line_old)-1]:#новое значение функции цели больше старого-первый элемент имеет индекс 0, второй - индекс 1
            raise Exception("Новый оптимум оказался больше предыдущего, хотя ищется минимум.")

    if Max_or_Min == 2:
        if Z_line[len(Z_line)-1] < Z_line_old[len(Z_line_old)-1]:#новое значение функции цели меньше старого
            raise Exception("Новый оптимум оказался меньше предыдущего, хотя ищется максимум.")

    return Z_line
def recount(matrix_input, Z_resh, str_resh): # пересчет мартрицы
    matrix = matrix_input.copy()

    matrix[str_resh] /= matrix[str_resh][Z_resh] # деление разрешающей строки на разрешающий элемент-чтобы сделать разрешающий элемент равным 1

    for i in range(len(matrix)):
        if i != str_resh:
            matrix[i] -= matrix[i][Z_resh] * matrix[str_resh] # правило прямоугольника

    return matrix
def solve(matrix, Z_line, basis):

    print(matrix)
    print("--------------------------------------")

    print(Z_line)
    print('Базис:',basis)

    print("Что ищем: 1 - минимум, 2 - максимум")
    Max_or_Min = int(input())

    if Max_or_Min != 1 and Max_or_Min != 2:
        Exception('Неверные данные')

    #Ставим в Z_line вместо М большое число

    if Max_or_Min == 1:
        for i in range(len(Z_line)):
            if hasattr(Z_line[i], 'evalf') == true:
                Z_line[i] = Z_line[i].evalf(subs={M: 10000.0})#-минимизация-вместо символа M в Z-строке ставится большое число

    if Max_or_Min == 2:
        for i in range(len(Z_line)):
            if hasattr(Z_line[i], 'evalf') == true:
                Z_line[i] = Z_line[i].evalf(subs={M: -10000.0})#-максимизация

    flag = Continue_Function(Z_line, Max_or_Min)

    n = 0
    while flag and n < 20:  # Критерий остановки/продолжения алгоритма
        n = n+1
        #Находится решающий столбец Z_resh
        #ищем максимальный элемент строки Z (решающий столбец). Нет смысла проверять на отрицательность всех элементов,
        #так как в таком случае flag просто выйдет из цикла.
        bufferZresh  = Z_line[0:len(Z_line) - 1] #элемент столбца B не учитывается

        if Max_or_Min == 1: # при минимумме
            Z_resh = np.argmax(bufferZresh) # максимальный положительный

        if Max_or_Min == 2: # при максимуме
            Z_resh = np.argmin(bufferZresh) # минимальный отрицательный

        print('Реш. сто.:',Z_resh)

        #поиск решающей строки-которая находит строку, в которой частное элементов столбца B и решающего столбца минимально и положительно.
        str_resh = get_str_resh(Z_resh, matrix)
        print('Реш. стр.:',str_resh)

        #вывод решающего элемента
        print('Решающий Элемент:',matrix[str_resh][Z_resh])

        #пересчёт Z строки
        Z_line = get_Z_line(matrix, Z_line, Z_resh, str_resh, Max_or_Min)

        #пересчёт матрицы
        matrix = recount(matrix, Z_resh, str_resh)
        print("______________________________________")
        print(np.around(matrix, decimals = 4))
        print("--------------------------------------")

        #пересчёт базиса-Обновляется базис, исключая старый столбец и добавляя новый столбец (соответствующий решающему элементу).
        basis[str_resh] = Z_resh

        print(Z_line)
        print('Базис:',basis)

        #пересчёт критерия остановки
        flag = Continue_Function(Z_line, Max_or_Min)

    return matrix, Z_line, basis
class M_metod:

    # Способ создания объекта (конструктор)
    def __init__(self, Z, X, B, Signs):# принимает на вход коэффициенты
        self.Z = Z
        self.X = X
        self.B = B
        self.Signs = Signs

    # Метод поиска оптимального плана.
    def find_optimal_plan(self):

        matrix, Z_line, basis = canonization(self.Z, self.X, self.B, self.Signs)

        matrix, Z_line, basis = solve(matrix, Z_line, basis)

        B = matrix[:, matrix.shape[1] - 1] # извлекается столбец "B" из матрицы matrix

        Rn = [] #столбцы с R
        Sn = [] #Все столбцы Sn ( и с -M в нижней части, и с 0)

        for i in range(Signs.size):
            if Signs[i] == '>=' or Signs[i] == '<=':
                Sn.append(i)
            if Signs[i] == '=' or Signs[i] == '>=':
                Rn.append(i)

        #в случае если в базисе есть R, то решение несовместно.
        for i in range(len(basis)):
                if basis[i] > (len(X[0]) + len(Sn) - 1):#это индекс последнего столбца, который соответствует последней переменной S.
                    raise Exception("Решение несовместно, так как в базисе остались R.")

        #удаление ненужных столбцов R
        for i in range(len(Rn)):
            matrix = np.delete(matrix, len(matrix[0])-2, 1)# удаляет предпоследний столбец из матрицы-R
            Z_line.pop(len(Z_line)-2)#удаление предпоследнего элемента из вектора Z_line

        print('|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|')
        print('Ответ:')
        print(matrix)
        print("--------------------------------------")
        print(Z_line)

        #вывод значений Xmax
        x = np.zeros(len(Z_line)-1)
        for i in range(len(basis)):
                x[basis[i]] = B[i]

        print("x = " + str(x))
        print("result = " + str(Z_line[len(Z_line)-1]))

from scipy.optimize import linprog

# Coefficients of the objective function to be minimized
c = [2, -5, 2, -1, -1]

# Coefficients of the inequality constraints (Ax <= b)
A = [
    [1, -14, 2, -1, -1],
    [10, -6, 9, -5, -2],
    [1, 3, 19, -10, -5]
]
b = [9, 2, 4]

# Bounds for variables
x_bounds = [(0, None)] * len(c)

# Solve the linear programming problem
result = linprog(c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')

# Check the success of the optimization
if result.success:
    print("Optimal solution found:")
    print("Objective Value:", result.fun)
    print("Optimal Solution:", result.x)
else:
    print("Optimization failed:", result.message)



#--------------------------------------------------- Вар. 1
'''
Z = np.array([2, -5, 2, -1, -1])
X = np.array([[1, -14, 2, -1, -1],
              [10, -6, 9, -5, -2],
              [1, 3, 19, -10, -5],], dtype= float)
B = np.array([9, 2, 4], dtype= float)
Signs = np.array(['=', '=', '='])
m_metod = M_metod(Z, X, B, Signs)

m_metod.find_optimal_plan()

'''
#--------------------------------------------------- Вар. 2

#Z = np.array([8, -2, 4, -7, 4])
#X = np.array([[4, -7, 6, 5, -2],
#              [9, 2, -9, 3, 9],
#              [-1, 5, 2, -4, 5],], dtype= float)
#B = np.array([9, 6, 6], dtype= float)
#Signs = np.array(['=', '=', '='])
#m_metod = M_metod(Z, X, B, Signs)

#m_metod.find_optimal_plan()

#--------------------------------------------------- Вар. 3

#Z = np.array([1, -8, 5, -4, 0])
#X = np.array([[5, -7, 5, 6, 9],
#              [0, 6, 0, 8, -9],
#              [0, 4, 3, -9, 2],], dtype= float)
#B = np.array([1, 6, 1], dtype= float)
#Signs = np.array(['=', '=', '='])
#m_metod = M_metod(Z, X, B, Signs)

#m_metod.find_optimal_plan()

#--------------------------------------------------- Вар. 8

Z = np.array([9, -8, 8, -9, -3])
X = np.array([[-5, 5, 7, 3, 8],
              [5, 3, -8, 3, 4],
              [1, 4, 2, -3, -4],], dtype= float)
B = np.array([1, 6, 8], dtype= float)
Signs = np.array(['=', '=', '='])
m_metod = M_metod(Z, X, B, Signs)

m_metod.find_optimal_plan()





#--------------------------------------------------- Вар. 5
#Z = np.array([2, -1, 6, -9, 3])
#X = np.array([[0, 5, -7, 5, 0],
#              [5, 2, 4, -6, 4],
#              [4, -8, 4, 3, 0],], dtype= float)
#B = np.array([4, 2, 3], dtype= float)
#Signs = np.array(['=', '=', '='])
#m_metod = M_metod(Z, X, B, Signs)

#m_metod.find_optimal_plan()
#--------------------------------------------------- Вар. 10
#Z = np.array([4, 0, 4, -5, 5])
#X = np.array([[1, 1, 8, 6, 6],
#              [-3, 7, 2, 1, -8],
#              [4, 8, -4, 2, 2],], dtype= float)
#B = np.array([4, 1, 1], dtype= float)
#Signs = np.array(['=', '=', '='])
#m_metod = M_metod(Z, X, B, Signs)

#m_metod.find_optimal_plan()
#--------------------------------------------------- Вар. 11
#Z = np.array([7, -5, 0, 0, 4])
#X = np.array([[-7, -7, 4, 2, 9],
#              [5, 6, 2, -7, 5],
#              [4, 1, 8, 3, -7],], dtype= float)
#B = np.array([8, 8, 7], dtype= float)
#Signs = np.array(['=', '=', '='])
#m_metod = M_metod(Z, X, B, Signs)

#m_metod.find_optimal_plan()